In [6]:
from owlready2 import *

onto1 = get_ontology("file:///home/michael/Desktop/CrosetWork/t3test2_no_semicolon_G_labelled.owl").load()

In [7]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [8]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto1.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = 'GO:' + cl[-7:]
    ids.append(clt)

In [9]:
#parsing biological process and molecular function classes

#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}
pilnary = {}
noparent = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []        
        

#remove anything that has no aspect or is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value.get('aspect') == None:
            keys.append(key)
            c+=1
    elif value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
        
print(c)

for i in keys:
    del nary[i]


for key, value in nary.items():
    if value.get('parent') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'parent' : value['parent']})
        pilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        noparent.update({key : filnary})

4367


In [10]:
#for pilnary

for key, value in pilnary.items():
    for item in value['parent']:
        if pilnary[item['id']].get('parent') == None:
            pilnary[item['id']]['parent'] = []
            if {'id' : key, 'relation' : item['relation']} not in pilnary[item['id']]['parent']:
                pilnary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            if {'id' : key, 'relation' : item['relation']} not in pilnary[item['id']]['parent']:
                pilnary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})

KeyError: 'GO:0005739'

In [11]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

#remove anything that has no aspect or is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value.get('aspect') == None:
            keys.append(key)
            c+=1
    elif value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
        
print(c)

for key, value in nary.items():
    for i in value['children']:
        if i['id'] in keys:
            c+=1
        elif value.get('babies') == None:
            value['babies'] = []
            value['babies'].append(i)
        else:
            value['babies'].append(i)
for i in keys:
    del nary[i]

0


In [14]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto1:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [15]:
reldict = {}

for key, value in pilnary.items():            
    for item in value['parent']:
        if reldict.get(item['relation']) == None:
            reldict[item['relation']] = [[],[]]
            propertymaker(item['relation'], 'void', 'void')
    for item in value['parent']:
        if item['id'] in ids:
            reldict[item['relation']][0].append(ids[ids.index(item['id'])])
            reldict[item['relation']][1].append(ids[ids.index(value['id'])])

In [16]:
#Making subproperties positively_regulates and negatively_regulates

with onto1:
    class regulates(ObjectProperty):
        pass
    class positively_regulates(regulates):
        pass
    class negatively_regulates(regulates):
        pass

In [17]:
#converting string to object in reldict lists

for key, value in reldict.items():
    for item in value[0]:
        value[0][value[0].index(item)] = getattr(onto1, ('G' + item[-7:]))
    for item in value[1]:
        value[1][value[1].index(item)] = getattr(onto1, ('G' + item[-7:]))


In [18]:
print(reldict['is_a'][0][:5], reldict['is_a'][1][:5])

[t3test2_no_semicolon_G_labelled.G0048308, t3test2_no_semicolon_G_labelled.G0048311, t3test2_no_semicolon_G_labelled.G0007005, t3test2_no_semicolon_G_labelled.G0008150, t3test2_no_semicolon_G_labelled.G0007582] [t3test2_no_semicolon_G_labelled.G0000001, t3test2_no_semicolon_G_labelled.G0000001, t3test2_no_semicolon_G_labelled.G0000002, t3test2_no_semicolon_G_labelled.G0000003, t3test2_no_semicolon_G_labelled.G0000003]


In [19]:
ded = {}

for key, value in reldict.items():
    for item in value[0]:
        #if the item dict does not exist make it
        if ded.get(value[0][value[0].index(item)]) == None:
            ded[value[0][value[0].index(item)]] = {}
            #if the key does not exist add the relation as key
            if ded[value[0][value[0].index(item)]].get(key) == None:
                ded[value[0][value[0].index(item)]][key] = []
                ded[value[0][value[0].index(item)]][key].append(value[1][value[0].index(item)])
            else:
                ded[value[0][value[0].index(item)]][key].append(value[1][value[0].index(item)])
        else:
            if ded[value[0][value[0].index(item)]].get(key) == None:
                ded[value[0][value[0].index(item)]][key] = []
                ded[value[0][value[0].index(item)]][key].append(value[1][value[0].index(item)])
            else:
                ded[value[0][value[0].index(item)]][key].append(value[1][value[0].index(item)])

In [20]:
ded

{t3test2_no_semicolon_G_labelled.G0048308: {'is_a': [t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001]},
 t3test2_no_semicolon_G_labelled.G0048311: {'is_a': [t3test2_no_semicolon_G_labelled.G0000001,
   t3test2_no_semicolon_G_labelled.G0000001]},
 t3test2_no_semicolon_G_labelled.G0007005: {'is_a': [t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G0000002,
   t3test2_no_semicolon_G_labelled.G

In [ ]:
for key, value in ded.items():
    

In [12]:
#setting relations

for key, value in ded.items():
    for key2, value2 in value.items():
        
        setattr(key, key2, value2)

In [16]:
reldict['part_of'][0]

[t3test2_no_semicolon_G.G0032042,
 t3test2_no_semicolon_G.G0003006,
 t3test2_no_semicolon_G.G0019098,
 t3test2_no_semicolon_G.G0022414,
 t3test2_no_semicolon_G.G0015151,
 t3test2_no_semicolon_G.G1905121,
 t3test2_no_semicolon_G.G0061804,
 t3test2_no_semicolon_G.G0051256,
 t3test2_no_semicolon_G.G0061805,
 t3test2_no_semicolon_G.G0032450,
 t3test2_no_semicolon_G.G0034463,
 t3test2_no_semicolon_G.G0034462,
 t3test2_no_semicolon_G.G0034463,
 t3test2_no_semicolon_G.G0046915,
 t3test2_no_semicolon_G.G0061736,
 t3test2_no_semicolon_G.G0061753,
 t3test2_no_semicolon_G.G0061739,
 t3test2_no_semicolon_G.G0034497,
 t3test2_no_semicolon_G.G0007080,
 t3test2_no_semicolon_G.G0000022,
 t3test2_no_semicolon_G.G0007076,
 t3test2_no_semicolon_G.G0007079,
 t3test2_no_semicolon_G.G0007064,
 t3test2_no_semicolon_G.G0007083,
 t3test2_no_semicolon_G.G0051306,
 t3test2_no_semicolon_G.G0072394,
 t3test2_no_semicolon_G.G0072395,
 t3test2_no_semicolon_G.G0072436,
 t3test2_no_semicolon_G.G0072437,
 t3test2_no_se

In [18]:
reldict['part_of'][1]

[t3test2_no_semicolon_G.G0000002,
 t3test2_no_semicolon_G.G0000003,
 t3test2_no_semicolon_G.G0000003,
 t3test2_no_semicolon_G.G0000003,
 t3test2_no_semicolon_G.G0000017,
 t3test2_no_semicolon_G.G0000022,
 t3test2_no_semicolon_G.G0000022,
 t3test2_no_semicolon_G.G0000022,
 t3test2_no_semicolon_G.G0000022,
 t3test2_no_semicolon_G.G0000023,
 t3test2_no_semicolon_G.G0000027,
 t3test2_no_semicolon_G.G0000028,
 t3test2_no_semicolon_G.G0000028,
 t3test2_no_semicolon_G.G0000041,
 t3test2_no_semicolon_G.G0000045,
 t3test2_no_semicolon_G.G0000045,
 t3test2_no_semicolon_G.G0000045,
 t3test2_no_semicolon_G.G0000045,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000070,
 t3test2_no_semicolon_G.G0000075,
 t3test2_no_semicolon_G.G0000075,
 t3test2_no_semicolon_G.G0000076,
 t3test2_no_semicolon_G.G0000076,
 t3test2_no_se

In [14]:
ded

{t3test2_no_semicolon_G.G0033955: {'part_of': [t3test2_no_semicolon_G.G0000266]},
 t3test2_no_semicolon_G.G0032504: {'is_a': [t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003]},
 t3test2_no_semicolon_G.G0019954: {'is_a': [t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003]},
 t3test2_no_semicolon_G.G0061887: {'is_a': [t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003]},
 t3test2_no_semicolon_G.G0019953: {'is_a': [t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0000003,
   t3test2_no_semicolon_G.G0

In [13]:
onto1.save(file='t3test2_no_semicolon_G_labelled_related_ded.owl', format = 'rdfxml')